In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon

%matplotlib inline

In [2]:
cost_min = pd.read_csv('../OCR/cost_min.csv')
paths_tot = pd.read_csv('../OCR/paths_tot.csv')


In [3]:
cost_min.head()

,fid,tid,cost,pass_avi,sent,cost_f
0,31021,31022,1.718334,no,1,1.72
1,31021,31023,3.118334,no,1,3.12
2,31021,31047,3.427223,no,1,3.43
3,31021,31048,3.124722,no,1,3.12
4,31021,31051,4.151681,yes,1,4.15


In [4]:
paths_tot.head()

,first,last,cat1,cat2,cat3,cat4,cat5,cat6,catTot,row_id
0,10000,10000,817.333,3.333,1.367,0.167,0.1,0.367,822.667,1
1,10000,10006,6.500,0.000,0.000,0.000,0.0,0.000,6.500,2
2,10000,3550308074,6.000,0.367,0.000,0.000,0.0,0.000,6.367,3
3,10000,3550308092,40.767,0.133,0.000,0.000,0.0,0.000,40.900,4
4,10000,3550308073,21.400,0.400,0.000,0.000,0.0,0.000,21.800,5


In [5]:
cost_min['from-to'] = cost_min['fid'].map(str) +'-'+cost_min['tid'].map(str)
cost_min.head()


,fid,tid,cost,pass_avi,sent,cost_f,from-to
0,31021,31022,1.718334,no,1,1.72,31021-31022
1,31021,31023,3.118334,no,1,3.12,31021-31023
2,31021,31047,3.427223,no,1,3.43,31021-31047
3,31021,31048,3.124722,no,1,3.12,31021-31048
4,31021,31051,4.151681,yes,1,4.15,31021-31051


In [6]:
paths_tot['first-last'] = paths_tot['first'].map(str) +'-'+paths_tot['last'].map(str)
paths_tot.head()

,first,last,cat1,cat2,cat3,cat4,cat5,cat6,catTot,row_id,first-last
0,10000,10000,817.333,3.333,1.367,0.167,0.1,0.367,822.667,1,10000-10000
1,10000,10006,6.500,0.000,0.000,0.000,0.0,0.000,6.500,2,10000-10006
2,10000,3550308074,6.000,0.367,0.000,0.000,0.0,0.000,6.367,3,10000-3550308074
3,10000,3550308092,40.767,0.133,0.000,0.000,0.0,0.000,40.900,4,10000-3550308092
4,10000,3550308073,21.400,0.400,0.000,0.000,0.0,0.000,21.800,5,10000-3550308073


In [7]:
cost_paths_tot = pd.merge(paths_tot,cost_min,how = 'inner', left_on = 'first-last', right_on = 'from-to' )
cost_paths_cars = cost_paths_tot[['row_id','first','last','cat1','cost_f']]
cost_paths_cars.head()

,row_id,first,last,cat1,cost_f
0,2,10000,10006,6.500,0.34
1,3,10000,3550308074,6.000,0.54
2,4,10000,3550308092,40.767,0.54
3,5,10000,3550308073,21.400,0.42
4,6,10000,3550308088,21.700,0.32


In [8]:
cost_paths_cars.describe()

,row_id,first,last,cat1,cost_f
count,573920.000000,5.739200e+05,5.739200e+05,573920.000000,573920.000000
mean,287468.946555,8.893241e+08,9.286134e+08,8.090446,1.216535
std,165973.265915,1.537391e+09,1.559338e+09,62.219018,1.187159
min,2.000000,1.000000e+04,1.000000e+04,0.000000,0.040000
25%,143724.750000,1.097200e+04,1.094300e+04,0.233000,0.460000
50%,287469.500000,1.282300e+04,1.291500e+04,0.767000,0.790000
75%,431222.250000,3.501608e+09,3.516408e+09,3.100000,1.470000
max,574916.000000,3.552809e+09,3.552809e+09,10269.930000,9.330000


In [9]:
cost_paths_cars.to_csv("cost_paths_cars.csv")

In [10]:
od_matrix = cost_paths_cars.pivot(index='first', columns='last', values='cat1')

In [11]:
nans = round(od_matrix.isnull().sum(axis=1)*100/od_matrix.shape[1],1)
nans_df = pd.DataFrame(nans, columns = ['Missing Values Rate'])
nans_df.to_excel("nans.xlsx", engine='xlsxwriter')

In [12]:
#pip install --upgrade xlsxwriter

In [14]:
od_matrix.to_csv("od_matrix.csv")